# Aviation Venture Risk EDA

## Introduction

ACME Co. is interested in purchasing and operating airplanes for commercial and private enterprises. This Exploratory Data Analysis (EDA) utilizes data from the National Transportation Safety Board to determine which aircraft have the lowest risk. The analysis contains actionable insights for the head of the new Aviation Division, Scott Fly.

## Import Python Libraries

In [877]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re #for spliting some values
import seaborn as sns
%matplotlib inline

# Load the Data

In [878]:
!ls data

AviationData.csv  USState_Codes.csv


In [879]:
#load the CSV files for the rest of the project

#pandas says columns 6, 7, and 28 have mixed data types.  
#For now we will set them to strings to avoid errors later
'''
Latitude and Longitude have two formats in the file.  One is degrees, minutes, seconds format 
with a suffix for hemisphere like N
The other format is called decimal degrees and it is a float.
'''

#latin1 is required as utf-8 will not load
#load 5 rows just for column names, a full load shows mixed data type warnings on columns 6, 7, 28
#so we will tell pandas to load them as strings
aviation_data = pd.read_csv("data/AviationData.csv",encoding="latin1", nrows=1)
col_list = list(aviation_data.columns)
dtype_spec = {
    col_list[6]: 'str', #Latitude
    col_list[7]: 'str', #Longitude
    col_list[28]: 'str' #Broad.phase.of.flight
}

#now load it in full without warnings
aviation_data = pd.read_csv("data/AviationData.csv",encoding="latin1", dtype=dtype_spec)
uscode_data = pd.read_csv("data/USState_Codes.csv")

# Standardize Column Names

In [880]:
aviation_data.columns = aviation_data.columns.str.lower().str.replace(' ', '_',regex=False).str.replace('.','_',regex=False)

# Info() and null count

In [881]:
#initial missing data check before cleaning/standardization
null_counts_before_standardization = aviation_data.isnull().sum()
aviation_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   event_id                88889 non-null  object 
 1   investigation_type      88889 non-null  object 
 2   accident_number         88889 non-null  object 
 3   event_date              88889 non-null  object 
 4   location                88837 non-null  object 
 5   country                 88663 non-null  object 
 6   latitude                34382 non-null  object 
 7   longitude               34373 non-null  object 
 8   airport_code            50249 non-null  object 
 9   airport_name            52790 non-null  object 
 10  injury_severity         87889 non-null  object 
 11  aircraft_damage         85695 non-null  object 
 12  aircraft_category       32287 non-null  object 
 13  registration_number     87572 non-null  object 
 14  make                    88826 non-null

# Mixed Data Type Issue with Longitude, Latitude

For latitude and longitude they mix formats.  Some are in degrees, minutes and seconds format with a suffix like 'N' for direction.  Some are in decimal degrees, which are easier to work with mathematically.

In [882]:
#To know to pass str types for columns 6, 7 and 28 we had to know what is up with those columns
#We do value_counts to see if one issue comes up a lot and inspect some initial values
aviation_data['latitude'].value_counts()

332739N      19
335219N      18
334118N      17
32.815556    17
324934N      16
             ..
039613N       1
342034N       1
433113N       1
343255N       1
373829N       1
Name: latitude, Length: 25589, dtype: int64

In [883]:
aviation_data['longitude'].value_counts()

0112457W       24
1114342W       18
1151140W       17
-104.673056    17
-112.0825      16
               ..
0843135W        1
0101957W        1
1064131W        1
1114414W        1
0121410W        1
Name: longitude, Length: 27154, dtype: int64

## mixed data type for broad_phase_of_flight 
This is likely due to NaN, but needs more investigation

In [884]:
aviation_data['broad_phase_of_flight'].head()

0      Cruise
1     Unknown
2      Cruise
3      Cruise
4    Approach
Name: broad_phase_of_flight, dtype: object

In [885]:
aviation_data['broad_phase_of_flight'].value_counts()

Landing        15428
Takeoff        12493
Cruise         10269
Maneuvering     8144
Approach        6546
Climb           2034
Taxi            1958
Descent         1887
Go-around       1353
Standing         945
Unknown          548
Other            119
Name: broad_phase_of_flight, dtype: int64

In [886]:
aviation_data['broad_phase_of_flight'].isna().sum()

27165

In [887]:
aviation_data[aviation_data['broad_phase_of_flight'].isna()]['broad_phase_of_flight'].head()

3030    NaN
3550    NaN
3637    NaN
4032    NaN
5505    NaN
Name: broad_phase_of_flight, dtype: object

# Standardize Data

Lets attack getting rid of duplicate labels.  We checked ahead and there's a lot of labels duplicated for a lot of reasons.  This attempts to get rid of all of that.

In [888]:
#lowercase and stripe white space, this removes a lot of duplicate labels
def standardize_string(s):
    if pd.isna(s):
        return s
    if isinstance(s,str):
        s = s.strip().lower()#.replace(' ','_')
    return s

#for some columns treating none as nan helps us do analysis
def none_to_None(s):
    if s == 'none':
        return None #None is the NaN for object columns
    return s

def unknown_to_None(s):
    if s == 'unknown':
        return None
    return s

#for some columns unk is short for unknown but unknown will still be listed, so standardize it
def unk_to_unknown(s):
    if s == 'unk':
        return 'unknown'
    return s

def replace_in_cols(cols,map):
    aviation_data[cols] = aviation_data[cols].applymap(map)


#display(aviation_data.head())
#we are excluding latitude and longitude bc capital N instead of lowercase n is how it is supposed to be
exclude_cols = ['latitude','longitude']
#an array of True and False that will have False for the columns we put in exclude_cols
include_mask = ~aviation_data.columns.isin(exclude_cols)
#loc is used here to target the dataframe columns with the include mask.
#say latitude is the 4th column, the array would have False for the 4th column and that would
#mean pandas doesn't select the 4th column.
#this is just easier than listing all the columns but 2
aviation_data.loc[:,include_mask] = aviation_data.loc[:,include_mask].applymap(standardize_string)

replace_in_cols(['airport_code','airport_name','registration_number'], none_to_None)
replace_in_cols(['aircraft_category','registration_number','engine_type','far_description'], unk_to_unknown)
replace_in_cols(['engine_type','aircraft_damage','registration_number'], unknown_to_None)

#make date strings into datetimes
aviation_data['publication_date'] = pd.to_datetime(aviation_data['publication_date'],format='%d-%m-%Y')

aviation_data.head()

,event_id,investigation_type,accident_number,event_date,location,country,latitude,longitude,airport_code,airport_name,...,purpose_of_flight,air_carrier,total_fatal_injuries,total_serious_injuries,total_minor_injuries,total_uninjured,weather_condition,broad_phase_of_flight,report_status,publication_date
0,20001218x45444,accident,sea87la080,1948-10-24,"moose creek, id",united states,NaN,NaN,NaN,NaN,...,personal,NaN,2.0,0.0,0.0,0.0,unk,cruise,probable cause,NaT
1,20001218x45447,accident,lax94la336,1962-07-19,"bridgeport, ca",united states,NaN,NaN,NaN,NaN,...,personal,NaN,4.0,0.0,0.0,0.0,unk,unknown,probable cause,1996-09-19
2,20061025x01555,accident,nyc07la005,1974-08-30,"saltville, va",united states,36.922223,-81.878056,NaN,NaN,...,personal,NaN,3.0,NaN,NaN,NaN,imc,cruise,probable cause,2007-02-26
3,20001218x45448,accident,lax96la321,1977-06-19,"eureka, ca",united states,NaN,NaN,NaN,NaN,...,personal,NaN,2.0,0.0,0.0,0.0,imc,cruise,probable cause,2000-09-12
4,20041105x01764,accident,chi79fa064,1979-08-02,"canton, oh",united states,NaN,NaN,NaN,NaN,...,personal,NaN,1.0,2.0,NaN,0.0,vmc,approach,probable cause,1980-04-16


## Split injury_severity

In [889]:
#split the fatal(123) format into two diff columns
severity = []
number = []
for row in aviation_data['injury_severity']:
    if isinstance(row,str):
        match = re.match(r"(\w+)\((\d+)\)", row)
        if match:
            severity.append(match.group(1))
            number.append(int(match.group(2)))
        else:
            severity.append(row)
            number.append(pd.NA)
    else:
        severity.append(None)
        number.append(pd.NA)

aviation_data['injury_severity_type'] = severity
aviation_data['injury_severity_count'] = number

#converts to int col 
aviation_data['injury_severity_count'] = aviation_data['injury_severity_count'].astype('Int64')

#aviation_data.head()
display(aviation_data[['injury_severity_type','injury_severity_count']])
display(aviation_data['injury_severity_count'])

,injury_severity_type,injury_severity_count
0,fatal,2
1,fatal,4
2,fatal,3
3,fatal,2
4,fatal,1
...,...,...
88884,minor,<NA>
88885,None,<NA>
88886,non-fatal,<NA>
88887,None,<NA>


0           2
1           4
2           3
3           2
4           1
         ... 
88884    <NA>
88885    <NA>
88886    <NA>
88887    <NA>
88888    <NA>
Name: injury_severity_count, Length: 88889, dtype: Int64

## State codes

In [890]:
state_codes = tuple(uscode_data['Abbreviation'].values)
display(uscode_data)
def extract_state(location):
    if pd.notnull(location) and ', ' in location:
        code = location.split(', ')[1].strip().upper()
        if code in state_codes:
            return code
    return None

#split the states into its own column but only if it's 'united states'
#np.where(condition, value if condition is true, value if condition is false)
aviation_data['state'] = np.where(
    aviation_data['country'] == 'united states',
    aviation_data['location'].apply(extract_state),
    None
)

aviation_data['state'].value_counts()

,US_State,Abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA
...,...,...
57,Virgin Islands,VI
58,Washington_DC,DC
59,Gulf of mexico,GM
60,Atlantic ocean,AO


CA    8857
TX    5913
FL    5825
AK    5672
AZ    2834
CO    2726
WA    2612
IL    2060
MI    2030
GA    2023
NY    1914
OH    1823
PA    1790
OR    1772
NC    1661
MO    1565
WI    1560
AR    1519
MN    1456
ID    1436
NM    1358
UT    1336
IN    1320
VA    1275
OK    1240
NV    1236
LA    1216
NJ    1170
AL    1153
TN    1108
KS    1104
MT    1050
SC     974
MA     969
IA     819
MD     815
MS     813
WY     739
NE     725
KY     650
ND     561
ME     505
CT     502
HI     481
SD     446
WV     394
NH     368
VT     241
RI     158
DE     114
PR     114
GM      44
DC      42
AO      17
PO      14
GU       8
VI       6
Name: state, dtype: int64

## Aviation code check

In [891]:
aviation_data['airport_code'].value_counts()

pvt     497
apa     160
ord     149
mri     137
den     115
       ... 
78oh      1
0v6       1
8c3       1
56m       1
eikh      1
Name: airport_code, Length: 10345, dtype: int64

## Fix private variations in airport names

In [892]:
replacements = {
    'private airstrip': 'private',
    'private strip': 'private'
}
aviation_data['airport_name'] = aviation_data['airport_name'].replace(replacements)
aviation_data['airport_name'].value_counts()

private                           898
merrill field                     109
centennial                        102
van nuys                           97
north las vegas                    82
                                 ... 
lambert-st. louis int'l             1
williamson mingo cty                1
sanona creek airstrip               1
penns cave                          1
wichita dwight d eisenhower nt      1
Name: airport_name, Length: 21563, dtype: int64

## Compare nulls sum after standardization

In [893]:
null_counts_after_standardization = aviation_data.isnull().sum()
comparison = pd.DataFrame({
    'Before Standardization': null_counts_before_standardization,
    'After Standardization': null_counts_after_standardization
})
#replace_in_cols(['airport_code','airport_name','registration_number'], none_to_nan)
#replace_in_cols(['aircraft_category','registration_number','engine_type','far_description'], unk_to_unknown)
#replace_in_cols(['engine_type','aircraft_damage','registration_number'], unknown_to_nan)
comparison_filtered = comparison[comparison['Before Standardization'] != comparison['After Standardization']]
comparison_filtered

,Before Standardization,After Standardization
aircraft_damage,3194.0,3313
airport_code,38640.0,40252
airport_name,36099.0,36331
engine_type,7077.0,9129
injury_severity_count,NaN,76325
injury_severity_type,NaN,1000
registration_number,1317.0,1752
state,NaN,6756


It looks like we're getting more nulls which is what we wanted in some cases!

In [894]:
aviation_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 34 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   event_id                88889 non-null  object        
 1   investigation_type      88889 non-null  object        
 2   accident_number         88889 non-null  object        
 3   event_date              88889 non-null  object        
 4   location                88837 non-null  object        
 5   country                 88663 non-null  object        
 6   latitude                34382 non-null  object        
 7   longitude               34373 non-null  object        
 8   airport_code            48637 non-null  object        
 9   airport_name            52558 non-null  object        
 10  injury_severity         87889 non-null  object        
 11  aircraft_damage         85576 non-null  object        
 12  aircraft_category       32287 non-null  object

## Far Description

The 'far_description' has a considerably smaller portion of missing data, compared to columns such as 'purpose_of_flight'. Let's investigate these unique values, to see if we can use them to fill other parts of our missing data. 

We'll organize the values as we go, to make them easier to group. 

In [895]:
aviation_data['far_description'].unique()

array([nan, 'part 129: foreign', 'part 91: general aviation',
       'part 135: air taxi & commuter', 'part 125: 20+ pax,6000+ lbs',
       'part 121: air carrier', 'part 137: agricultural',
       'part 133: rotorcraft ext. load', 'unknown',
       'part 91f: special flt ops.', 'non-u.s., non-commercial',
       'public aircraft', 'non-u.s., commercial', 'public use',
       'armed forces', 'part 91 subpart k: fractional', '091', 'nusc',
       '135', 'nusn', '121', '137', '129', '133', '091k', 'pubu', 'armf',
       '103', '125', '437', '107'], dtype=object)

**129** and **part 129: foreign**: Foreign air carriers or operators who would like to conduct passenger or cargo operations to, from, and transiting the US. A foreign air carrier is a person, not a citizen of the U.S., undertaking by any means to provide foreign air transportation. 

**121** and **part 121: air carrier**: Generally large, U.S.-based airlines, regional air carriers, and all cargo operarters.

**non-u.s., commercial**: non-u.s., commercial

**091** and **part 91: general aviation**: General aviation pilots, operating private aircraft. Non-commercial situations only.

**091k** and **part 91 subpart k: fractional**: For private, not-for-hire use by individuals and companies

**135** and **part 135: air taxi & commuter**: For commuter and on-demand operations (flying passengers on-demand, rather than on a scheduled-basis). Can vary from small single aircraft operators, to large operators providing a network to move cargo to larger carriers. Often offer critical services to remote areas.

**125** and **part 125: 20+ pax,6000+ lbs**: Large group charter flight (non-commercial)

**133** and **part 133: rotorcraft ext. load**: On demand, external load air service, where air carriers utilize helicopters

**137** and **part 137: agricultural**: Dispensing any economic poison, other substance or activities dispensed for agriculture, horticulture, or forest preservation.

**public aircraft**: For governmental use only. 

**public use**: "Public" aircrafts are aircrafts used only by the government, however this label could also be referring to a "public use" airport, which would be for commercial purposes. 

**armed forces**: armed forces

**unknown**: unknown

**part 91f: special flt ops.**: Non-commercial flights only.

**non-u.s., non-commercial'**: non-u.s., non-commercial

**nusc**: nusc

**nusn**: nusn

**pubu**: pubu

**armf**: armf

**103**: Ultraleight vehicles. Purely recreational flights.

**437**: Experimental permit using a suborbital rocket

**107**: Small unmanned aircraft systems

In [896]:
#Lets standardize FAR
# Define the function to standardize the far_description
def standardize_description(description):
    if pd.isna(description):
        return description
    if isinstance(description,str):
        description = description.lstrip('0').strip()
        #description = description.replace('subpart k','k').strip()
    if 'part' in description:
        match = re.search(r'part (\d+):', description)
        if match:
            return match.group(1).strip()
    
    return description

# Apply the function row-wise
aviation_data['far_description'] = aviation_data['far_description'].apply(standardize_description)

display(aviation_data['far_description'].value_counts())
display(aviation_data['far_description'].isna().sum())

91                               24707
nusn                              1584
137                               1447
135                               1044
nusc                              1013
121                                844
unknown                            393
129                                346
pubu                               253
133                                139
non-u.s., non-commercial            97
non-u.s., commercial                93
public use                          19
91k                                 14
125                                 10
armf                                 8
107                                  4
public aircraft                      2
103                                  2
armed forces                         1
part 91 subpart k: fractional        1
part 91f: special flt ops.           1
437                                  1
Name: far_description, dtype: int64

56866

# Checking for Missing Data
First we will run some checks on the aviation_data to see what we're dealing with for missing data

In [769]:
aviation_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 34 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   event_id                88889 non-null  object        
 1   investigation_type      88889 non-null  object        
 2   accident_number         88889 non-null  object        
 3   event_date              88889 non-null  object        
 4   location                88837 non-null  object        
 5   country                 88663 non-null  object        
 6   latitude                34382 non-null  object        
 7   longitude               34373 non-null  object        
 8   airport_code            48637 non-null  object        
 9   airport_name            52558 non-null  object        
 10  injury_severity         87889 non-null  object        
 11  aircraft_damage         85576 non-null  object        
 12  aircraft_category       32287 non-null  object

It seems we are missing a lot of data!  We will need to formulate plans on all of this.
This next code block will let us see how much data is missing percent wise

## Missing Percent

In [770]:
# Calculate the percentage of missing values for each column
missing_perc = aviation_data.isna().mean() * 100
missing_perc

event_id                   0.000000
investigation_type         0.000000
accident_number            0.000000
event_date                 0.000000
location                   0.058500
country                    0.254250
latitude                  61.320298
longitude                 61.330423
airport_code              45.283443
airport_name              40.872324
injury_severity            1.124999
aircraft_damage            3.727120
aircraft_category         63.677170
registration_number        1.970998
make                       0.070875
model                      0.103500
amateur_built              0.114750
number_of_engines          6.844491
engine_type               10.270112
far_description           63.974170
schedule                  85.845268
purpose_of_flight          6.965991
air_carrier               81.271023
total_fatal_injuries      12.826109
total_serious_injuries    14.073732
total_minor_injuries      13.424608
total_uninjured            6.650992
weather_condition          5

Convention suggests dropping columns where over 50% of the data is missing, unless it is very important to your analysis.
These are candidates to consider dropping:

In [771]:
missing_perc[missing_perc > 50]

latitude                 61.320298
longitude                61.330423
aircraft_category        63.677170
far_description          63.974170
schedule                 85.845268
air_carrier              81.271023
injury_severity_count    85.865518
dtype: float64

## location

Lets see what Location has, maybe it will help us decide on what to do with Lat and Long

In [772]:
aviation_data['location'].value_counts()

anchorage, ak       548
miami, fl           275
houston, tx         271
albuquerque, nm     265
chicago, il         256
                   ... 
medina, mn            1
circle pines, mn      1
pine island, fl       1
churchtown, oh        1
brasnorte,            1
Name: location, Length: 21977, dtype: int64

We have options now, we can drop Lat and Long or we can use some sort of API to get the Lat and Long filled based on the Locatoin when it is missing!  We don't have to decide now, lets let further exploration guide our choices.

# Exploring More Columns

## investigation_type

In [773]:
aviation_data['investigation_type'].value_counts()

accident    85015
incident     3874
Name: investigation_type, dtype: int64

Accident - caused personal injury, damage or damage
Incident - could have potentially caused harm but didn't necessarily 

Weighting these differently is a good idea.

## country

In [774]:
aviation_data['country'].value_counts()

united states     82248
brazil              374
canada              359
mexico              358
united kingdom      344
                  ...  
chad                  1
ivory coast           1
cambodia              1
yemen                 1
benin                 1
Name: country, Length: 215, dtype: int64

There is a strong bias to the United States so our data is only going to be the most reliable in the US due to uncontrolled factors like regulation differences between countries.

## airport_code and airport_name

In [775]:
aviation_data['airport_code'].value_counts()

pvt     497
apa     160
ord     149
mri     137
den     115
       ... 
78oh      1
0v6       1
8c3       1
56m       1
eikh      1
Name: airport_code, Length: 10345, dtype: int64

In [776]:
aviation_data['airport_name'].value_counts()

private                           898
merrill field                     109
centennial                        102
van nuys                           97
north las vegas                    82
                                 ... 
lambert-st. louis int'l             1
williamson mingo cty                1
sanona creek airstrip               1
penns cave                          1
wichita dwight d eisenhower nt      1
Name: airport_name, Length: 21563, dtype: int64

We will have to clean the names quite a bit especially around private airports.
The codes have a lot of missing data could that correlate with private airports not having codes?


In [777]:
aviation_data[(aviation_data['airport_name']=='private') & (aviation_data['airport_code'].isna())][['airport_name','airport_code']]

,airport_name,airport_code
64,private,NaN
182,private,NaN
225,private,NaN
270,private,NaN
730,private,NaN
...,...,...
87442,private,None
87470,private,NaN
87887,private,NaN
88425,private,NaN


In [778]:
aviation_data[(aviation_data['airport_name']=='private') & (aviation_data['airport_code'].notna())][['airport_code']].value_counts()

airport_code
pvt             182
priv              6
xxx               4
unk               2
rla               2
0                 1
x                 1
unkn              1
rrdi              1
nowr              1
n\a               1
my99              1
-                 1
ipy               1
999               1
4nc5              1
2az1              1
172               1
12                1
0co6              1
0c09              1
04il              1
00al              1
mm20              1
dtype: int64

In [779]:
aviation_data[aviation_data['airport_name']=='private']['airport_name'].count()

898

In [780]:
aviation_data['injury_severity'].value_counts()

non-fatal     67357
fatal(1)       6167
fatal          5262
fatal(2)       3711
incident       2219
              ...  
fatal(270)        1
fatal(60)         1
fatal(43)         1
fatal(143)        1
fatal(230)        1
Name: injury_severity, Length: 109, dtype: int64

We will need to seperate fatal from the number to let it make more sense for our charts and analysis.

## aircraft_damage

In [781]:
aviation_data['aircraft_damage'].value_counts()

substantial    64148
destroyed      18623
minor           2805
Name: aircraft_damage, dtype: int64

Finally a field that has little issues, just a few missing values.

## Aircraft Specs
Lets examine the group of fields that might let us identify features of the airplanes, some of the most important data

### aircraft_category

In [782]:
aviation_data['aircraft_category'].value_counts()

airplane             27617
helicopter            3440
glider                 508
balloon                231
gyrocraft              173
weight-shift           161
powered parachute       91
ultralight              30
unknown                 16
wsft                     9
powered-lift             5
blimp                    4
rocket                   1
ultr                     1
Name: aircraft_category, dtype: int64

We can get rid of all that aren't airplane, probably

### registration_number

In [783]:
aviation_data['registration_number'].value_counts()

unreg     131
usaf        9
n20752      8
n53893      6
n11vh       6
         ... 
n62951      1
n1013e      1
n8266r      1
n65737      1
n9026p      1
Name: registration_number, Length: 79091, dtype: int64

Lots of NONE might not be as useful a field.  However if some planes were just train wrecks and had lots of issues then they could be 
something worth keeping in mind when interpretting the data
### make

In [784]:
aviation_data['make'].value_counts()

cessna           27149
piper            14870
beech             5372
boeing            2745
bell              2722
                 ...  
cohen                1
kitchens             1
lutes                1
izatt                1
royse ralph l        1
Name: make, Length: 7587, dtype: int64

We will need to clean this data for sure the Cessna for example is listed twice with diff capitilization
### model

In [785]:
aviation_data['model'].value_counts()

152                 2367
172                 1756
172n                1164
pa-28-140            932
150                  829
                    ... 
e75nl                  1
747-273c               1
watcha-mccall-it       1
md-520n                1
m-8 eagle              1
Name: model, Length: 11646, dtype: int64

Make and Model together can form unique strings to identify types of craft so this is nice.  (It is possible for a model to have the same name and diff makes, even if rare).  This pair will be used in a lot of finding the correlations in the data.
## amateur_built

In [786]:
aviation_data['amateur_built'].value_counts()

no     80312
yes     8475
Name: amateur_built, dtype: int64

Since we are a business we might just remove rows about amateur built aircraft.  However maybe some of the non business made ones have properties we'd be interested in?

In [787]:
aviation_data['number_of_engines'].value_counts()

1.0    69582
2.0    11079
0.0     1226
3.0      483
4.0      431
8.0        3
6.0        1
Name: number_of_engines, dtype: int64

I'm wondering what has zero engines, gliders perhaps?  Lets find out below

In [788]:
aviation_data[aviation_data['number_of_engines']==0][['engine_type']].value_counts()

engine_type  
none             10
reciprocating     2
dtype: int64

It seems it's mostly missing data.  Also our missing data is going to need standardized to one value.

In [789]:
aviation_data['engine_type'].value_counts()

reciprocating      69530
turbo shaft         3609
turbo prop          3391
turbo fan           2481
turbo jet            703
none                  21
geared turbofan       12
electric              10
lr                     2
hybrid rocket          1
Name: engine_type, dtype: int64

## far_description
FAR stands for Federal Aviation Regulations. These regulations are a comprehensive set of rules and guidelines established by the Federal Aviation Administration (FAA) to ensure the safety and efficiency of civil aviation within the United States.

Some examples:
- Part 23: Airworthiness Standards for Normal, Utility, Acrobatic, and Commuter Category Airplanes
- Part 61: Certification: Pilots, Flight Instructors, and Ground Instructors
- Part 91: General Operating and Flight Rules
- Part 121: Operating Requirements: Domestic, Flag, and Supplemental Operations

In [790]:
aviation_data['far_description'].value_counts()

91                               24707
nusn                              1584
137                               1447
135                               1044
nusc                              1013
121                                844
unknown                            393
129                                346
pubu                               253
133                                139
non-u.s., non-commercial            97
non-u.s., commercial                93
public use                          19
91k                                 14
125                                 10
armf                                 8
107                                  4
public aircraft                      2
103                                  2
armed forces                         1
part 91 subpart k: fractional        1
part 91f: special flt ops.           1
437                                  1
Name: far_description, dtype: int64

The Data for this will need standardized as seen by the Part 91 and etc.  
on part 91 and others
https://pilotinstitute.com/part-91-vs-121-vs-135/

NUSN: Non-U.S. National (incident or accident)

"NUSN" is a code used to identify incidents or accidents involving aircraft that are not registered in the United States. This categorization helps in distinguishing between domestic and international incidents for regulatory and statistical purposes.
NUSC: Non-U.S. Commercial (incident or accident)

"NUSC" refers to incidents or accidents involving non-U.S. commercial aircraft. This code is specifically used for commercial operations, such as airlines and charter services, that are not registered in the United States.

... Understanding all the codes is going to be important if we use this field

## schedule

In [791]:
aviation_data['schedule'].value_counts()

nsch    4474
unk     4099
schd    4009
Name: schedule, dtype: int64

NSCH (Non-Scheduled): Refers to flights that do not operate on a regular schedule. These can include charter flights, air taxi operations, private flights, and other ad-hoc operations.

UNK (Unknown): Indicates that the schedule type of the flight operation is unknown. This can occur when the information is not available or not recorded in the safety database.

SCHD (Scheduled): Refers to flights that operate on a regular, published schedule. These are typically commercial airline flights that follow a fixed timetable.

## purpose_of_flight

In [792]:
aviation_data['purpose_of_flight'].value_counts()

personal                     49448
instructional                10601
unknown                       6802
aerial application            4712
business                      4018
positioning                   1646
other work use                1264
ferry                          812
aerial observation             794
public aircraft                720
executive/corporate            553
flight test                    405
skydiving                      182
external load                  123
public aircraft - federal      105
banner tow                     101
air race show                   99
public aircraft - local         74
public aircraft - state         64
air race/show                   59
glider tow                      53
firefighting                    40
air drop                        11
asho                             6
pubs                             4
publ                             1
Name: purpose_of_flight, dtype: int64

This is very informative as a field and it seems to have minimal issues with missing data.  Here is the Skydiving Joseph mentioned.

## air_carrier

In [793]:
aviation_data['air_carrier'].value_counts()

pilot                         258
american airlines              90
united airlines                89
delta air lines                53
delta air lines inc            44
                             ... 
frank w. scooley                1
richard l. mcglashan            1
inflight pilot traning llc      1
mills & daughters inc           1
mc cessna 210n llc              1
Name: air_carrier, Length: 13208, dtype: int64

It has a lot of missing rows but it could have a big impact on safety.

## total_fatal_injuries

In [794]:
aviation_data['total_fatal_injuries'].value_counts()

0.0      59675
1.0       8883
2.0       5173
3.0       1589
4.0       1103
         ...  
156.0        1
68.0         1
31.0         1
115.0        1
176.0        1
Name: total_fatal_injuries, Length: 125, dtype: int64

it is in floats for some reason so we might make sure there's no "partial fatal injuries"

## total_serious_injuries

In [795]:
aviation_data['total_serious_injuries'].value_counts()

0.0      63289
1.0       9125
2.0       2815
3.0        629
4.0        258
5.0         78
6.0         41
7.0         27
9.0         16
10.0        13
8.0         13
13.0         9
11.0         6
26.0         5
14.0         5
12.0         5
20.0         3
25.0         3
28.0         3
21.0         2
59.0         2
50.0         2
17.0         2
47.0         2
137.0        1
19.0         1
161.0        1
27.0         1
35.0         1
67.0         1
33.0         1
88.0         1
125.0        1
53.0         1
34.0         1
41.0         1
18.0         1
63.0         1
55.0         1
23.0         1
43.0         1
39.0         1
45.0         1
44.0         1
16.0         1
60.0         1
106.0        1
81.0         1
15.0         1
22.0         1
Name: total_serious_injuries, dtype: int64

It is in floats again
## total_minor_injuries

In [796]:
aviation_data['total_minor_injuries'].value_counts()

0.0      61454
1.0      10320
2.0       3576
3.0        784
4.0        372
5.0        129
6.0         67
7.0         59
9.0         22
8.0         20
13.0        14
10.0        11
12.0        11
14.0        10
11.0         9
17.0         8
19.0         6
18.0         6
24.0         5
22.0         5
25.0         4
16.0         4
15.0         4
33.0         4
20.0         3
21.0         3
26.0         3
23.0         3
32.0         3
27.0         3
50.0         2
30.0         2
36.0         2
31.0         2
28.0         2
42.0         2
38.0         2
57.0         1
65.0         1
84.0         1
43.0         1
35.0         1
380.0        1
47.0         1
68.0         1
200.0        1
71.0         1
58.0         1
171.0        1
39.0         1
96.0         1
29.0         1
69.0         1
62.0         1
45.0         1
125.0        1
40.0         1
Name: total_minor_injuries, dtype: int64

It is in floats again
## total_uninjured

In [797]:
aviation_data['total_uninjured'].value_counts()

0.0      29879
1.0      25101
2.0      15988
3.0       4313
4.0       2662
         ...  
558.0        1
412.0        1
338.0        1
401.0        1
455.0        1
Name: total_uninjured, Length: 379, dtype: int64

also in floats
## weather_condition

In [798]:
aviation_data['weather_condition'].value_counts()

vmc    77303
imc     5976
unk     1118
Name: weather_condition, dtype: int64

UNK and Unk are not standardized

Conditions are codes:

**VMC (Visual Meteorological Conditions):**

VMC: Indicates weather conditions that allow for visual flight rules (VFR) operations. Pilots can navigate and control the aircraft using visual references outside the cockpit.

**IMC (Instrument Meteorological Conditions):**

IMC: Refers to weather conditions that require instrument flight rules (IFR) operations. Visibility and cloud cover are such that pilots must rely on cockpit instruments for navigation and control.

**UNK (Unknown):**

UNK: Indicates that the meteorological conditions at the time of the incident or data point are unknown or not recorded.


## broad_phase_of_flight

In [799]:
aviation_data['broad_phase_of_flight'].value_counts()

landing        15428
takeoff        12493
cruise         10269
maneuvering     8144
approach        6546
climb           2034
taxi            1958
descent         1887
go-around       1353
standing         945
unknown          548
other            119
Name: broad_phase_of_flight, dtype: int64

Landing is the most prone to incident or accident it seems like, especially since it is also relatively short in terms of phases.
This data can help us find planes weak points and strong points as well as high risk areas.

## report_status

In [800]:
aviation_data['report_status'].value_counts()

probable cause                                                                                                                                                                                                                                      61754
foreign                                                                                                                                                                                                                                              1999
<br /><br />                                                                                                                                                                                                                                          167
factual                                                                                                                                                                                                                                               145


So it seems a lot of the values are probable cause

**Probable Cause**: This term is used to describe the findings of an investigation that identify the factors or events that most likely led to the incident or accident. When a report reaches the "probable cause" status, the investigating authority, such as the National Transportation Safety Board (NTSB) in the United States, has completed its analysis and has determined the primary reasons behind the occurrence.


## publication_date

In [801]:
aviation_data['publication_date'].value_counts()

2020-09-25    17019
2020-09-26     1769
2020-11-03     1155
1993-03-31      452
2003-11-25      396
              ...  
2004-11-29        1
2001-08-29        1
2004-11-18        1
1996-12-17        1
2022-12-29        1
Name: publication_date, Length: 2924, dtype: int64

# Describe and Identify Outliers

In [ ]:
aviation_data.describe()

,number_of_engines,total_fatal_injuries,total_serious_injuries,total_minor_injuries,total_uninjured,injury_severity_count
count,82805.000000,77488.000000,76379.000000,76956.000000,82977.000000,12564.000000
mean,1.146585,0.647855,0.279881,0.357061,5.325440,2.834846
std,0.446510,5.485960,1.544084,2.235625,27.913634,11.234181
min,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,1.000000,0.000000,0.000000,0.000000,1.000000,2.000000
75%,1.000000,0.000000,0.000000,0.000000,2.000000,2.000000
max,8.000000,349.000000,161.000000,380.000000,699.000000,349.000000


Looking at the description of the numerical data in our dataset, there are a few things of note...

**The first three quartiles** of total_fatal_injuries, total_serious_injuries, and total_minor_injuries, are all equal to zero. Yet the **max** of these three categories are comparably high numbers. A similar data skew is true for 'total_uninjured'. These are outliers that can either be helpful or hurtful to consider in our analysis.

In this instance, outliers are particularly useful because they point to aircrafts with higher cases of risk.

Outliers mean that the best measure of central tendency to use when filling in missing values is probably the **median**

# Filtering Data

There is a lot duplicate and unecessary data. Before we can figure out how to best group the items in our dataset, we need to filter out some of the unecessary information

In [802]:
aviation_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 34 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   event_id                88889 non-null  object        
 1   investigation_type      88889 non-null  object        
 2   accident_number         88889 non-null  object        
 3   event_date              88889 non-null  object        
 4   location                88837 non-null  object        
 5   country                 88663 non-null  object        
 6   latitude                34382 non-null  object        
 7   longitude               34373 non-null  object        
 8   airport_code            48637 non-null  object        
 9   airport_name            52558 non-null  object        
 10  injury_severity         87889 non-null  object        
 11  aircraft_damage         85576 non-null  object        
 12  aircraft_category       32287 non-null  object

#### Geographic Location

**Longitude and Latitude**: We don't need 'longitude' and 'latitude', we can just use 'location' column for our findings.

In [803]:
#Define a function creating a tuple of latitude and longitude. 
"""
def latitude_and_longitude(row):
    if pd.isnull(row['location']):
        return (row['latitude'], row['longitude'])
    else:
        return row['location']

#Make a new column with the 'latitude' and 'longitude' coordinates as a tuple. 
aviation_data['lat_and_long'] = aviation_data.apply(latitude_and_longitude, axis=1)

#Fill missing values in 'location' with the tuples in the new 'lat_and_long' column. 
aviation_data['location'] = aviation_data['location'].fillna(aviation_data['lat_and_long'])

#Drop the 'lat_and_long' column
aviation_data.drop(columns=['lat_and_long'], inplace=True)
"""

"\ndef latitude_and_longitude(row):\n    if pd.isnull(row['location']):\n        return (row['latitude'], row['longitude'])\n    else:\n        return row['location']\n\n#Make a new column with the 'latitude' and 'longitude' coordinates as a tuple. \naviation_data['lat_and_long'] = aviation_data.apply(latitude_and_longitude, axis=1)\n\n#Fill missing values in 'location' with the tuples in the new 'lat_and_long' column. \naviation_data['location'] = aviation_data['location'].fillna(aviation_data['lat_and_long'])\n\n#Drop the 'lat_and_long' column\naviation_data.drop(columns=['lat_and_long'], inplace=True)\n"

In [804]:
#Drop the latitude and longitude columns
aviation_data.drop(columns=['latitude', 'longitude'], inplace=True)

#### Date and Time

**Publication Date**: The publication date is also uncessary. We have other time markers that will be far more relevant to the aim of our investigation.

In [805]:
#'publication_date' is unecessary to our analysis.
#Error when reformatting notebook, so must add blank column, to be able to note dropping it
#aviation_data['publication_date'] = 0
aviation_data.drop(columns=['publication_date'], inplace=True)

#### Summary Statistics - Injury Severity

We need to find a way to quanitify the injuries per accident. To do so, we have to further categorize and clean the data.

In [806]:
#Investigating necessity of columns

print('Unique categories in injury_severity:', aviation_data['injury_severity'].unique())
print('Unique categories in injury_severity_type:', aviation_data['injury_severity_type'].unique())
print('Unique categories in injury_severity_count:', aviation_data['injury_severity_count'].unique())

Unique categories in injury_severity: ['fatal(2)' 'fatal(4)' 'fatal(3)' 'fatal(1)' 'non-fatal' 'incident'
 'fatal(8)' 'fatal(78)' 'fatal(7)' 'fatal(6)' 'fatal(5)' 'fatal(153)'
 'fatal(12)' 'fatal(14)' 'fatal(23)' 'fatal(10)' 'fatal(11)' 'fatal(9)'
 'fatal(17)' 'fatal(13)' 'fatal(29)' 'fatal(70)' 'unavailable'
 'fatal(135)' 'fatal(31)' 'fatal(256)' 'fatal(25)' 'fatal(82)'
 'fatal(156)' 'fatal(28)' 'fatal(18)' 'fatal(43)' 'fatal(15)' 'fatal(270)'
 'fatal(144)' 'fatal(174)' 'fatal(111)' 'fatal(131)' 'fatal(20)'
 'fatal(73)' 'fatal(27)' 'fatal(34)' 'fatal(87)' 'fatal(30)' 'fatal(16)'
 'fatal(47)' 'fatal(56)' 'fatal(37)' 'fatal(132)' 'fatal(68)' 'fatal(54)'
 'fatal(52)' 'fatal(65)' 'fatal(72)' 'fatal(160)' 'fatal(189)'
 'fatal(123)' 'fatal(33)' 'fatal(110)' 'fatal(230)' 'fatal(97)'
 'fatal(349)' 'fatal(125)' 'fatal(35)' 'fatal(228)' 'fatal(75)'
 'fatal(104)' 'fatal(229)' 'fatal(80)' 'fatal(217)' 'fatal(169)'
 'fatal(88)' 'fatal(19)' 'fatal(60)' 'fatal(113)' 'fatal(143)' 'fatal(83)'
 'fatal(

Before diving in deeper...Since we made our injury severity 'type' and 'count' columns, we can drop 'injury_severity'.

In [807]:
#Dropping column
aviation_data.drop(columns=['injury_severity'], inplace=True)

## Make and Model

Due to the outliers in the data, we know that our mean will be skewed. So we should probably use the median to fill in these missing values. 
However, to have an even more detailed analysis, we can **group the accidents by make and model** and filter imputations through these groupings. Then we can imputate the missing values using the **mean** of these groupings.

Before we do so, we must deal with the missing data in these columns. 

Let's remind ourselves of how much data is missing in the 'make' and 'model' columns.

In [809]:
aviation_data['make'].isna().sum()

63

In [810]:
aviation_data['model'].isna().sum()

92

Since there aren't that many missing values, we can just drop those rows.

In [811]:
aviation_data.dropna(subset=['make', 'model'], inplace=True)

print('Number of missing values in make:', aviation_data['make'].isna().sum())
print('Number of missing values in mode:', aviation_data['model'].isna().sum())

Number of missing values in make: 0
Number of missing values in mode: 0


## Concatenate Make and Model

Create a new column that specifies an aircrafts 'make_and_model', accounting for different model types across the same make.

In [812]:
aviation_data['make_model'] = aviation_data['make'] + '-' + aviation_data['model']
display(aviation_data.head())
display(aviation_data['make_model'].isna().sum())
display(aviation_data['make_model'].value_counts())

,event_id,investigation_type,accident_number,event_date,location,country,airport_code,airport_name,aircraft_damage,aircraft_category,...,total_serious_injuries,total_minor_injuries,total_uninjured,weather_condition,broad_phase_of_flight,report_status,injury_severity_type,injury_severity_count,state,make_model
0,20001218x45444,accident,sea87la080,1948-10-24,"moose creek, id",united states,NaN,NaN,destroyed,NaN,...,0.0,0.0,0.0,unk,cruise,probable cause,fatal,2,ID,stinson-108-3
1,20001218x45447,accident,lax94la336,1962-07-19,"bridgeport, ca",united states,NaN,NaN,destroyed,NaN,...,0.0,0.0,0.0,unk,unknown,probable cause,fatal,4,CA,piper-pa24-180
2,20061025x01555,accident,nyc07la005,1974-08-30,"saltville, va",united states,NaN,NaN,destroyed,NaN,...,NaN,NaN,NaN,imc,cruise,probable cause,fatal,3,VA,cessna-172m
3,20001218x45448,accident,lax96la321,1977-06-19,"eureka, ca",united states,NaN,NaN,destroyed,NaN,...,0.0,0.0,0.0,imc,cruise,probable cause,fatal,2,CA,rockwell-112
4,20041105x01764,accident,chi79fa064,1979-08-02,"canton, oh",united states,NaN,NaN,destroyed,NaN,...,2.0,NaN,0.0,vmc,approach,probable cause,fatal,1,OH,cessna-501


0

cessna-152                          2366
cessna-172                          1753
cessna-172n                         1163
piper-pa-28-140                      932
cessna-150                           829
                                    ... 
john w. culp-kr-2                      1
barrigar-rv-6                          1
pohlman/pohlman-rotorway exec 90       1
boeing-737-200rs                       1
royse ralph l-glasair                  1
Name: make_model, Length: 18245, dtype: int64

## Set 0 to nan for number of engines

In [813]:
aviation_data.loc[aviation_data['number_of_engines'] == 0, 'number_of_engines'] = np.nan

## Cleaning Data Types

Before we go on, let's make all our numerical objects the same data type if possible.

Let's check whether or not we need float objects, as opposed to integer objects, by filtering out floats that end with a decimal point other than ".0"

This will ensure there are no aircrafts built with entities such as "half-engines", or accidents with "half-fatalities".

### Show float columns

In [814]:
aviation_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88777 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   event_id                88777 non-null  object 
 1   investigation_type      88777 non-null  object 
 2   accident_number         88777 non-null  object 
 3   event_date              88777 non-null  object 
 4   location                88725 non-null  object 
 5   country                 88552 non-null  object 
 6   airport_code            48602 non-null  object 
 7   airport_name            52514 non-null  object 
 8   aircraft_damage         85486 non-null  object 
 9   aircraft_category       32245 non-null  object 
 10  registration_number     87075 non-null  object 
 11  make                    88777 non-null  object 
 12  model                   88777 non-null  object 
 13  amateur_built           88678 non-null  object 
 14  number_of_engines       81528 non-null

### Find floats in float cols or can they all be int?

In [815]:

float_cols = aviation_data.select_dtypes(include=['float64']).columns
display(float_cols)

#uncomment to check code, but then rerun all up to this cell when done
#aviation_data['number_of_engines'] = 1.1
# Iterate over each float column
for col in float_cols:
    print(f"Column: {col}")
    # Check and print values that are not integers
    #it will return a series of True and False based on if it's both not nan and float != int of the same number
    non_int_values = aviation_data[col][aviation_data[col].apply(lambda x: not np.isnan(x) and x != int(x))]
    if not non_int_values.empty:
        print("Non-integer values:")
        print(non_int_values)
    else:
        print("All values are integers.")
    print()

Index(['number_of_engines', 'total_fatal_injuries', 'total_serious_injuries',
       'total_minor_injuries', 'total_uninjured'],
      dtype='object')

Column: number_of_engines
All values are integers.

Column: total_fatal_injuries
All values are integers.

Column: total_serious_injuries
All values are integers.

Column: total_minor_injuries
All values are integers.

Column: total_uninjured
All values are integers.



### Convert cols to Int64 which allows null values

In [816]:
#since they are all integers lets convert the column
for col in float_cols:
    aviation_data[col] = aviation_data[col].astype('Int64')
aviation_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88777 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   event_id                88777 non-null  object
 1   investigation_type      88777 non-null  object
 2   accident_number         88777 non-null  object
 3   event_date              88777 non-null  object
 4   location                88725 non-null  object
 5   country                 88552 non-null  object
 6   airport_code            48602 non-null  object
 7   airport_name            52514 non-null  object
 8   aircraft_damage         85486 non-null  object
 9   aircraft_category       32245 non-null  object
 10  registration_number     87075 non-null  object
 11  make                    88777 non-null  object
 12  model                   88777 non-null  object
 13  amateur_built           88678 non-null  object
 14  number_of_engines       81528 non-null  Int64 
 15  en

# Impute

## Impute Aircraft Features based on make_model
some pre output to make sure it works

In [817]:
aviation_data[aviation_data['make_model']=='cessna-310r']['number_of_engines'].value_counts()

2    110
1      2
Name: number_of_engines, dtype: Int64

In [818]:
aviation_data[aviation_data['make_model']=='cessna-310r']['engine_type'].isna().sum()

10

In [819]:
aviation_data[aviation_data['make_model']=='cessna-310r']['engine_type'].value_counts()

reciprocating    109
Name: engine_type, dtype: int64

In [820]:
aviation_data['aircraft_category'].value_counts()

airplane             27580
helicopter            3435
glider                 508
balloon                231
gyrocraft              173
weight-shift           161
powered parachute       91
ultralight              30
unknown                 16
wsft                     9
powered-lift             5
blimp                    4
rocket                   1
ultr                     1
Name: aircraft_category, dtype: int64

In [821]:
aviation_data['aircraft_category'].isna().sum()

56532

This cell is computationally expensive, but it's been optimized twice

In [822]:
columns_to_impute = ['number_of_engines','engine_type','aircraft_category']

def impute_with_mode(row):
    #row is a series, and mode is the most common value in that series
    mode = row.mode() 
    if not mode.empty:
        #if we found a mode for this groupby series then impute it!
        return row.fillna(mode[0]) 
    return row

#group by model and save it so we don't recompute this every loop
groupby = aviation_data.groupby('make_model')

#loop over the cols we wish to impute the mode based on make_model
for col in columns_to_impute:
    #passes the groupby series for col for make_model pair to impute_with_mode
    #then assigns the result of that to aviation_data
    aviation_data[col] = groupby[col].transform(impute_with_mode)

Testing it worked!

In [823]:
aviation_data[aviation_data['make_model']=='cessna-310r']['number_of_engines'].value_counts()

2    117
1      2
Name: number_of_engines, dtype: Int64

In [824]:
aviation_data[aviation_data['make_model']=='cessna-310r']['engine_type'].isna().sum()

0

In [825]:
aviation_data[aviation_data['make_model']=='cessna-310r']['engine_type'].value_counts()

reciprocating    119
Name: engine_type, dtype: int64

The number of rows imputed just knowing a make and model is elsewhere in the data is huuuuuuge

In [826]:
aviation_data['aircraft_category'].value_counts()

airplane             67921
helicopter            7236
glider                 881
balloon                376
gyrocraft              200
weight-shift           162
powered parachute       92
ultralight              32
unknown                 16
wsft                     9
powered-lift             6
blimp                    5
rocket                   1
ultr                     1
Name: aircraft_category, dtype: int64

In [827]:
aviation_data['aircraft_category'].isna().sum()

11839

## Purpose of Flight

Next, we can fill a lot of the missing values of 'purpose_of_flight' using the lists of our 'far_description' values. We don't need to get too specific, since we're really trying to compare the risk associated with commercial vs. private aircraft use.

In [ ]:
aviation_data['purpose_of_flight'].unique()

array([nan, 'personal', 'business', 'instructional', 'ferry', 'unknown',
       'executive/corporate', 'aerial observation', 'aerial application',
       'public aircraft', 'skydiving', 'positioning', 'other work use',
       'public aircraft - federal', 'air race/show', 'flight test',
       'public aircraft - state', 'glider tow', 'banner tow',
       'firefighting', 'external load', 'air race show',
       'public aircraft - local', 'air drop', 'pubs', 'asho'],
      dtype=object)

In [ ]:
aviation_data['purpose_of_flight'].isna().sum()

3730

In [ ]:
#Defining lists by breaking the 'far_description' values into categories 'commercial' and 'private' use
commercial = ['129', '121', 'non-u.s., commercial']
private_91 = ['91', '91k', '91f']
private = ['135', '125', '103', 'non-u.s., non-commercial', '437', '107']
government = ['armed forces', 'public aircraft']
external_load = ['133']

#Defining a function to fill the missing values in 'purpose_of_flight', using these lists
def fill_purpose_flight(pf, far):
    if pd.isna(pf):
        if far in commercial:
            return 'commercial'
        elif far in private_91:
            return 'private_91'
        elif far in private:
            return 'private'
        elif far in government:
            return 'government'
        elif far in external_load:
            return 'external load'
    else:
        return pf

aviation_data['purpose_of_flight'] = aviation_data.apply(lambda x: fill_purpose_flight(x['purpose_of_flight'], x['far_description']), axis=1)

In [ ]:
aviation_data['purpose_of_flight'].isna().sum()

2126

In [ ]:
aviation_data.isna().sum()

event_id                      0
investigation_type            0
accident_number               0
event_date                    0
location                      0
country                       7
airport_code              10165
airport_name               9396
aircraft_damage            1379
aircraft_category             0
registration_number         254
make                          0
model                         0
amateur_built                 0
number_of_engines           693
engine_type                1012
far_description             499
schedule                  24598
purpose_of_flight          2126
air_carrier               16323
total_fatal_injuries       3159
total_serious_injuries     3216
total_minor_injuries       2871
total_uninjured             894
weather_condition          3044
broad_phase_of_flight     21195
report_status              4962
injury_severity_type      26643
injury_severity_count     26643
state                      3433
make_and_model                0
dtype: i

# Post Imputation Cleanup and Categorization

## Cleanup Aircraft Category

We filled in a lot of missing values for aircraft_category.  Now lets use that to narrow our data set down to what we want to work with for this project which is only airplanes!

In [828]:
aviation_data['aircraft_category'].unique()

array(['airplane', nan, 'helicopter', 'glider', 'balloon', 'gyrocraft',
       'ultralight', 'unknown', 'weight-shift', 'blimp', 'powered-lift',
       'powered parachute', 'rocket', 'wsft', 'ultr'], dtype=object)

We've already established that our company is looking to purchase aircrafts for large budgeted commercial purposes. So aircrafts other than 'airplanes' are irrelevant to our analysis. 

Through or exploratory research, we've found some 'make' values, or manufacturers, that are connected to the 'balloons' category. Let's drop these to be thorough, then continue examining our 'aircraft_category' column.

In [829]:
#Aicrafts with 'make' values that imply aircraft is a balloon
print(aviation_data[aviation_data['make'].str.contains('balloon', case=False)]['make'])

353               balloon works
746               balloon works
951               avian balloon
1112     thunder balloons, ltd.
1277              balloon works
                  ...          
87276      cameron balloons u s
87314       lindstrand balloons
87671        adams balloons llc
87889          cameron balloons
88476          firefly balloons
Name: make, Length: 267, dtype: object


In [830]:
#Getting sum of these values
aviation_data['make'].str.contains('balloon', case=False).sum()

267

In [831]:
#Dropping these values
#aviation_data.drop(aviation_data[aviation_data['make'].str.contains('balloon', case=False)].index, inplace=True)

In [832]:
aviation_data[aviation_data['make'].str.contains('balloon',case=False)]['aircraft_category'].value_counts()

balloon    179
Name: aircraft_category, dtype: int64

In [834]:
#assign them all to balloon category
aviation_data.loc[aviation_data['make'].str.contains('balloon',case=False),'aircraft_category'] = 'balloon'

In [839]:
aviation_data[aviation_data['make'].str.contains('balloon',case=False)]['aircraft_category'].value_counts()

balloon    267
Name: aircraft_category, dtype: int64

Back to our **aircraft categories**! 

In [742]:
#Double checking for an error where 'nan' shows up
nan_category_rows = aviation_data[aviation_data['aircraft_category'] == 'nan']
len(nan_category_rows)

0

Now let's define the few categories we're unfamiliar with, to be sure we don't need them. Then we can drop all the categories that are niether 'airplane' nor 'unknown'...

**'wsft'**: Looks like an aircraft used primarily in the *United Kingdom*, for private use...The *Piper PA-23-250* 

**'ultr'**: Seems to be another abbrevation for the 'ultralight' category already defined.

Neither of these are necessary to our data. Let's drop all of the unecessary information.

Before we do so, let's check how many airplanes we have in our dataset, to make sure we have plenty of information.

We'll also check the number of unknowns, and keep those for now.

In [838]:
num_airplane_rows = aviation_data[aviation_data['aircraft_category'] == 'airplane'].shape[0]
num_unknown_category_rows = aviation_data[aviation_data['aircraft_category'] == 'unknown'].shape[0]
print("Number of airplanes:", num_airplane_rows)
print("")
print("Number of unknown:", num_unknown_category_rows)
display(aviation_data['aircraft_category'].value_counts())

Number of airplanes: 67921

Number of unknown: 16


airplane             67921
helicopter            7236
glider                 881
balloon                464
gyrocraft              200
weight-shift           162
powered parachute       92
ultralight              32
unknown                 16
wsft                     9
powered-lift             6
blimp                    5
rocket                   1
ultr                     1
Name: aircraft_category, dtype: int64

Now we can use aircraft_category to answer questions!

## Percentage of Injuries

To make our numbers more digestable and easier to plot in visualizations...let's start grouping the number and types of injuries in our dataset.

First, we'll make a new column with 'total_injuries', by combining the number of serious and fatal injuries in each accident. 

We can make an additional column to calculate the percentage injured.

In [ ]:
#Make new column 'total_injuries'
aviation_data['total_injuries'] = aviation_data['total_fatal_injuries'] + aviation_data['total_serious_injuries']

#Calculate percentage of serious injuries
aviation_data['percentage_injured'] = aviation_data['total_injuries'] / (aviation_data['total_injuries'] + aviation_data['total_minor_injuries'] + aviation_data['total_uninjured'])
aviation_data['percentage_injured'] = aviation_data['percentage_injured'] * 100

In [ ]:
aviation_data.describe()

,number_of_engines,total_fatal_injuries,total_serious_injuries,total_minor_injuries,total_uninjured,injury_severity_count,total_injuries,percentage_injured
count,26887.000000,24421.000000,24364.000000,24709.000000,26686.000000,937.000000,24165.000000,23260.000000
mean,1.189980,0.655747,0.297734,0.254118,6.373567,4.055496,0.884544,31.501116
std,0.445396,5.946702,2.099036,2.897034,31.867330,15.104590,6.060590,44.557533
min,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,1.000000,2.000000,0.000000,0.000000
75%,1.000000,0.000000,0.000000,0.000000,2.000000,3.000000,1.000000,100.000000
max,8.000000,295.000000,161.000000,380.000000,588.000000,228.000000,295.000000,100.000000


### Fatal to Non-Fatal Ratio

We can also make a column outlining the ratio of fatal to non-fatal injuries.

In [844]:
#Add a new column with the fatal to non-fatal ratio
aviation_data['fatal_to_non_ratio'] = aviation_data['total_fatal_injuries']/(aviation_data['total_serious_injuries'] + aviation_data['total_minor_injuries'] + aviation_data['total_uninjured'])

#Make sure you account for places where the denominator might be 0
#aviation_data['fatal_to_non_ratio'].fillna(aviation_data['total_fatal_injuries'], inplace=True)

#As well as places where the numerator and denominator are both 0
#aviation_data['fatal_to_non_ratio'].fillna(0, inplace=True)

display(aviation_data['fatal_to_non_ratio'].value_counts())

0.0         58550
inf         12062
1.0          1308
0.5           320
2.0           235
            ...  
0.0625          1
0.571429        1
0.007194        1
112.0           1
48.5            1
Name: fatal_to_non_ratio, Length: 136, dtype: Int64

## Categorize Severity and create severity_category
We can make a new column that groups Fatal or Serious from all the other accident types
since our data is only accidents then we don't know how many successful flights there were
but we can use this column to indicate relative safety

In [849]:
def categorize_severity(severity):
    if severity in ['fatal', 'serious']:
        return 'Fatal or Serious'
    else:
        return 'Minor'

aviation_data['severity_category'] = aviation_data['injury_severity_type'].apply(categorize_severity)
display(aviation_data['injury_severity_type'].value_counts())
display(aviation_data['severity_category'].isna().sum())
display(aviation_data['severity_category'].value_counts())
display(aviation_data['severity_category'])

non-fatal      67305
fatal          17793
incident        2214
minor            218
serious          173
unavailable       95
Name: injury_severity_type, dtype: int64

0

Minor               70811
Fatal or Serious    17966
Name: severity_category, dtype: int64

0        Fatal or Serious
1        Fatal or Serious
2        Fatal or Serious
3        Fatal or Serious
4        Fatal or Serious
               ...       
88884               Minor
88885               Minor
88886               Minor
88887               Minor
88888               Minor
Name: severity_category, Length: 88777, dtype: object

# Questions (Business Understanding Section)

## Question 1: What is the safest category of flight?

## Question 2: What is the safest class of airplane travel?
Here we show that private flight is FAR more risky than commercial

## Question 3: What are the safest features of an airplane?
'amateur_built', 'number_of_engines', 'engine_type', 'far_description', 'schedule', 'purpose_of_flight'

In [444]:
cols_to_compare = ['amateur_built', 'number_of_engines', 'engine_type', 'far_description', 'schedule', 'purpose_of_flight']

## Question 4: What is the safest airplane?
Here we show what is the safest airplane

## Question 5: What is the safest location for airflight?
Here we show what is the safest location for flying

## Question 6: What is the safest airport for airflight?
Here we show what the safest airport is for airplanes

# 3 Questions 
(missing data with injury counts for all)

1) What is the safest category of flight? (private or commercial) 

2) What are the safest features of an aircraft? (engine type and number) (more missing data) 

3) What is the safest make/manufacturer to buy from?  Bonus what is the safest model (no missing data)